# Lab02: Frequent itemset mining

- Student ID: 20127458
- Student name: Đặng Tiến Đạt

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [494]:
from collections import defaultdict

### Read data
First we have to read data from database

In [495]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) # a python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
    
    return data, s

### Tree Projection

**I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**TODO:**

In [496]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)

    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    # TODO (hint: this function will be called in generateSearchSpace method.):
    return a + b

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}

        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        # TODO: why should we do this, answer it at the markdown below?
        
        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()

    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}

        search_space = {}
        for item, support in self.s.items():
            search_space[item] = {}

            search_space[item]['itemset'] = [item]
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.

            search_space[item]['pruned'] = False
            # TODO:
            # After finish implementing the algorithm tell me why should you use this
            # instead of delete item directly from search_space and tree.

            search_space[item]['support'] = support

            tree[item] = {}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.

            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            '''

        return tree, search_space

    def computeItemsetSupport(self, itemset):

        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        support = 0
        for tid, itemset_value in self.data.items():
            if set(itemset).issubset(itemset_value):
                support += 1
        return support

    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 0:
            return search_space[itter_node]['support']
        subtree = search_space[itter_node]
        for node in subtree.keys():
            k-=1
            self.get_sub_tree(k,tree,search_space,node)


    def prune(self, k, tree, search_space):

        '''
        In this method we will find out which itemset in current search space is frequent
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        if self.L.get(k) is None: self.L[k] = []
        # TODO
        # First, find out frequent itemsets in search_space and add them to L[k]
        # Second, prune those are not frequent itemsets.
        for node in search_space.keys():
            # check if node is frequent itemset >= minSup ?
            if search_space[node]['support'] >= self.minSup:
                self.L[k].append(search_space[node]['itemset'])
            else:
                # prune node when it is not frequent itemset
                search_space[node]['pruned'] = True
                tree[node] = {}


    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l = len(items)
        self.prune(k, tree, search_space)
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            if search_space[a]['pruned']: continue
            
            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
              
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).

                # TODO:
                # First create newset using join set
                newset = joinset(search_space[a]['itemset'], search_space[b]['itemset'])
                
                # Second add newset to search_space
                search_space[a][b]['itemset'] = newset
                search_space[a][b]['pruned'] = False
                search_space[a][b]['support'] = self.computeItemsetSupport(newset)
                
                sub_search_space[b] = search_space[a][b]
                sub_tree[b] = tree[a][b]
        
            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)

    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)

    def miningResults(self):
        return self.L

Ok, let's test on a typical dataset `chess`.

In [497]:
data, s= readData('chess.txt')

In [498]:
#
a=TP(data=data,s=s, minSup=3000)
tree_tp = a.miningResults()
print(tree_tp)

{1: [[48], [56], [66], [34], [62], [7], [36], [60], [40], [29], [52], [58]], 2: [[48, 52], [48, 58], [56, 29], [56, 52], [56, 58], [66, 60], [66, 29], [66, 52], [66, 58], [34, 40], [34, 29], [34, 52], [34, 58], [62, 60], [62, 40], [62, 29], [62, 52], [62, 58], [7, 60], [7, 40], [7, 29], [7, 52], [7, 58], [36, 60], [36, 40], [36, 29], [36, 52], [36, 58], [60, 40], [60, 29], [60, 52], [60, 58], [40, 29], [40, 52], [40, 58], [29, 52], [29, 58], [52, 58]], 3: [[48, 52, 48, 58], [56, 29, 56, 52], [56, 29, 56, 58], [56, 52, 56, 58], [66, 60, 66, 29], [66, 60, 66, 52], [66, 60, 66, 58], [66, 29, 66, 52], [66, 29, 66, 58], [66, 52, 66, 58], [34, 40, 34, 29], [34, 40, 34, 52], [34, 40, 34, 58], [34, 29, 34, 52], [34, 29, 34, 58], [34, 52, 34, 58], [62, 60, 62, 29], [62, 60, 62, 52], [62, 60, 62, 58], [62, 40, 62, 29], [62, 40, 62, 52], [62, 40, 62, 58], [62, 29, 62, 52], [62, 29, 62, 58], [62, 52, 62, 58], [7, 60, 7, 40], [7, 60, 7, 29], [7, 60, 7, 52], [7, 60, 7, 58], [7, 40, 7, 29], [7, 40, 7

### Answer questions here:
**Why don't we compute support of items while reading data?**
- To avoid reading data twice. We can compute support of items while reading data but we have to read data twice. The first time is to compute support of items and the second time is to compute support of itemsets. So we can compute support of items while reading data but we have to read data twice. So we don't compute support of items while reading data.
- Because over memory 
- Run time is too long 

**why should we do sort**
- This is done to make sure that the itemsets with lower support values are processed first, which reduces the size of the search space and the time complexity of the algorithm.If we process the itemsets with higher support values first, we would end up with a larger search space, and the algorithm would take more time to complete.
- We need to sort the items in each transaction in ascending order. This is because we need to use the items in each transaction as a key to search in the tree. If we don't sort the items, the key will be different even though the items are the same. \
For example, if we have a transaction {1, 2, 3}, the key will be 123 if we don't sort the items, but the key will be 321 if we sort the items. So we need to sort the items in each transaction in ascending order.

**study about python set and its advantages ?**
- Remove duplicate items from a list using set because set only contains unique elements.
- Implement perform common math operations like unions and intersections

**After finish implementing the algorithm tell me why should you use this? Instead of delete item directly from search_space and tree.**
- The itemset isn't frequent so we don't need to add it to the tree. We can delete it directly from search_space and tree.
- Maintain the tree and search_space is more efficient than delete item directly from search_space and tree.

**Apriori algorithm and Tree Projection, tell me the differences of two algorithms.** \
**Apriori Algorithm:**
- Uses a candidate generation and pruning approach to find frequent itemsets.
- Generates a large number of candidate itemsets and then prunes them based on the minimum support threshold.
- The algorithm uses a level-wise approach to generate frequent itemsets of size k by using frequent itemsets of size k-1.
- Suitable for mining frequent itemsets in transactional databases.
  
**Tree Projection Algorithm:**
- Uses a projection-based approach to find frequent itemsets.
- Constructs a conditional pattern base by projecting the database onto each frequent item and linking the projections together to form a tree structure.
- The algorithm recursively searches the tree to find all frequent itemsets.
- Suitable for mining frequent itemsets in databases with a large number of attributes.



# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). 

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  

Import library needed for this section.

In [499]:
from bs4 import BeautifulSoup
import pandas as pd

Define a function to get table data from file text

In [500]:
def read_and_parse_html(file_name):
   with open(file_name, 'r') as f:
      soup = BeautifulSoup(f, 'html.parser')
   return soup

def get_table(soup):
   table = soup.find_all('table')[0]
   data = []
   for tr in table.find_all('tr'):
      for td in tr.find_all('td'):
         data.append(td.text)
   data = [data[i] for i in range(len(data)) if i % 2 != 0]
   return data

def handle_data(data):
   header = data[0].split(',')
   data = [row.split(',') for row in data[1:]]
   return header, data

def get_data_frame(data):
   header, data = handle_data(data)
   df = pd.DataFrame(data, columns=header)
   return df

def get_table_df(file_name):
   soup = read_and_parse_html(file_name)
   table = get_table(soup)
   df = get_data_frame(table)
   return df

In [501]:
data = get_table_df('churn.txt')
data.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.100000,110,45.070000,...,99,16.780000,244.700000,91,11.010000,10.000000,3,2.700000,1,False.
1,OH,107,415,371-7191,no,yes,26,161.600000,123,27.470000,...,103,16.620000,254.400000,103,11.450000,13.700000,3,3.700000,1,False.
2,NJ,137,415,358-1921,no,no,0,243.400000,114,41.380000,...,110,10.300000,162.600000,104,7.320000,12.200000,5,3.290000,0,False.
3,OH,84,408,375-9999,yes,no,0,299.400000,71,50.900000,...,88,5.260000,196.900000,89,8.860000,6.600000,7,1.780000,2,False.
4,OK,75,415,330-6626,yes,no,0,166.700000,113,28.340000,...,122,12.610000,186.900000,121,8.410000,10.100000,3,2.730000,3,False.


Select the features that to be used for frequent itemset mining. At here, i used `Area Code`, `Int'l Plan`, `VMail Plan`, `Day Calls`, `Eve Calls`,`Night Calls`, `Intl Calls`, `CustServ Calls` and `Churn?` as features.

In [502]:
# choose the features
data_churn = data[['Area Code', 'Int\'l Plan', 'VMail Plan', 'Day Calls', 'Eve Calls', 'Night Calls', 'Intl Calls', 'CustServ Calls', 'Churn?']]
data_churn.head()

,Area Code,Int'l Plan,VMail Plan,Day Calls,Eve Calls,Night Calls,Intl Calls,CustServ Calls,Churn?
0,415,no,yes,110,99,91,3,1,False.
1,415,no,yes,123,103,103,3,1,False.
2,415,no,no,114,110,104,5,0,False.
3,408,yes,no,71,88,89,7,2,False.
4,415,yes,no,113,122,121,3,3,False.


transform data to binary with data type is category

In [503]:
data_churn['Churn?'] = data_churn['Churn?'].astype('category').cat.codes
data_churn['Int\'l Plan'] = data_churn['Int\'l Plan'].astype('category').cat.codes
data_churn['VMail Plan'] = data_churn['VMail Plan'].astype('category').cat.codes
data_churn.head()

/tmp/ipykernel_337/1296376074.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_churn['Churn?'] = data_churn['Churn?'].astype('category').cat.codes
/tmp/ipykernel_337/1296376074.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_churn['Int\'l Plan'] = data_churn['Int\'l Plan'].astype('category').cat.codes
/tmp/ipykernel_337/1296376074.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

,Area Code,Int'l Plan,VMail Plan,Day Calls,Eve Calls,Night Calls,Intl Calls,CustServ Calls,Churn?
0,415,0,1,110,99,91,3,1,0
1,415,0,1,123,103,103,3,1,0
2,415,0,0,114,110,104,5,0,0
3,408,1,0,71,88,89,7,2,0
4,415,1,0,113,122,121,3,3,0


Select the item that have churn = 1 -> the data that custormer who churn and then find the frequent itemset of this data 

In [504]:
# select Churn? = 1
data_churn = data_churn[data_churn['Churn?'] == 1]
data_churn.head()

,Area Code,Int'l Plan,VMail Plan,Day Calls,Eve Calls,Night Calls,Intl Calls,CustServ Calls,Churn?
10,415,0,0,137,83,111,6,4,1
15,415,0,0,67,97,128,9,4,1
21,408,0,0,89,121,64,6,5,1
33,408,0,0,118,119,90,3,1,1
41,408,1,1,85,107,78,15,0,1


In [505]:
# handle data_churn to itemset and support variable
def handle_data_churn(data_churn):
   s = data_churn.apply(pd.Series.value_counts).loc[1].to_dict()
   data = data_churn.apply(lambda row: set(row[row == 1].index), axis=1).to_dict()
   return data, s

# drop column Churn? because we need find frequent itemset of Churn? = 1
data_churn = data_churn.drop('Churn?', axis=1)

In [506]:
data_churn, s = handle_data_churn(data_churn)
minSup_churn = len(data_churn) * 0.05
tree_churn = TP(data_churn, s=s, minSup=minSup_churn).miningResults()
print(tree_churn)

{1: [['VMail Plan'], ["Int'l Plan"]], 2: [['VMail Plan', "Int'l Plan"]], 3: []}


=> We can see that the item that have high support and high confidence is the item that churn = 1

# 4 References

Feel free to send questions to my email address: ntthuhang0131@gmail.com
